In [1]:
import numpy as np
import pandas as pd
import os
from scipy import stats
import utils
import torch.optim as optim
import torch
from torch.utils.data.sampler import RandomSampler

import model.net as net
from dataloader import *
from train import train_and_evaluate

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [2]:
data_set = 'Irish_2010'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

100%|██████████| 918/918 [00:27<00:00, 32.90it/s]


In [3]:
month = 1
n_clusters = 2
method = 'hierarchical/euclidean'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)
path_data = os.path.join(path, 'data', 'deepar')

series = data[:, month-1, :months[month-1]*24]

weather = get_weather(path, data_set, month)
week = get_dow(data_set, month)
day = get_hod(month)

num_covariates = 4
covariates = np.zeros((num_covariates, len(series[0])))
covariates[1] = stats.zscore(weather)
covariates[2] = stats.zscore(week)
covariates[3] = stats.zscore(day)
covariates = covariates.T

In [6]:
for i in range(n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    
    test_data = sub_series[:, -168*2:].T
    train_data = sub_series[:, :-168].T
    
    data_start = (train_data != 0).argmax(axis=0)
    total_time = sub_series.shape[1]
    num_series = sub_series.shape[0]
    
    window_size = 192
    stride_size = 24
    
    # prepare data
    cov = covariates.copy()
    train_x_input, train_v_input, train_label = prep_data(train_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time)
    cov = covariates.copy()
    test_x_input, test_v_input, test_label = prep_data(test_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time, train=False)
    
    # params
    json_path = os.path.join(path, 'forecasting', 'deepar', 'params.json')
    params = utils.Params(json_path)
    
    params.relative_metrics = False
    params.sampling = False
    
    # use GPU if available
    cuda_exist = torch.cuda.is_available()
    # Set random seeds for reproducible experiments if necessary
    if cuda_exist:
        params.device = torch.device('cuda')
        # torch.cuda.manual_seed(240)
        model = net.Net(params).cuda()
    else:
        params.device = torch.device('cpu')
        # torch.manual_seed(230)
        model = net.Net(params)
    
    train_set = TrainDataset(train_x_input, train_label)
    test_set = TestDataset(test_x_input, test_v_input, test_label)
    sampler = WeightedSampler(train_v_input) # Use weighted sampler instead of random sampler
    train_loader = DataLoader(train_set, batch_size=params.batch_size, sampler=sampler, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=4)
    
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    loss_fn = net.loss_fn
    
    restore_file = None
    train_and_evaluate(model,
                       train_loader,
                       test_loader,
                       optimizer,
                       loss_fn,
                       params,
                       restore_file)
    break

begin training and evaluation
Epoch 1/100
train_loss: 1.0702459812164307
train_loss: 0.6449604034423828
train_loss: 0.6650168100992838
train_loss: 0.6394361654917399
train_loss: 0.639016588528951
train_loss: 0.6191946665445963
train_loss: 0.597570021947225
train_loss: 0.5824446280797323
train_loss: 0.5575777689615885
train_loss: 0.5133227904637655
train_loss: 0.5068854093551636
train_loss: 0.46700795491536456
train_loss: 0.45857715606689453
train_loss: 0.438839594523112
train_loss: 0.4471634229024251
train_loss: 0.2879294753074646
train_loss: 0.25182561079661053
train_loss: 0.25548962752024335
Current Best ND is: 0.64061
Epoch 2/100
train_loss: 0.2063016096750895
train_loss: 0.26644758383433026
train_loss: 0.1849398414293925
train_loss: 0.12452889482180278
train_loss: 0.18259602785110474
train_loss: 0.20975236097971597
train_loss: 0.127494345108668
train_loss: 0.07399355371793111
train_loss: 0.1098603109518687
train_loss: 0.09642035762468974
train_loss: 0.08278435468673706
train_loss: 

train_loss: -0.27539946635564166
train_loss: -0.34889670213063556
train_loss: -0.3337411085764567
train_loss: -0.20682334899902344
train_loss: -0.3107630213101705
train_loss: -0.31884392102559406
train_loss: -0.2742966612180074
train_loss: -0.17296463251113892
train_loss: -0.2677127917607625
train_loss: -0.26441073417663574
train_loss: -0.3081008195877075
train_loss: -0.32645291090011597
train_loss: -0.2860971490542094
train_loss: -0.26687244574228924
train_loss: -0.23664226134618124
train_loss: -0.30629334847132367
Current Best ND is: 0.62594
Epoch 15/100
train_loss: -0.2598416606585185
train_loss: -0.25453827778498334
train_loss: -0.33248021205266315
train_loss: -0.3938799301783244
train_loss: -0.2942809462547302
train_loss: -0.271628479162852
train_loss: -0.39035165309906006
train_loss: -0.2718234658241272
train_loss: -0.29577813545862836
train_loss: -0.22974437475204468
train_loss: -0.3228866060574849
train_loss: -0.29054681460062665
train_loss: -0.334042231241862
train_loss: -0.28

train_loss: -0.49101348718007404
train_loss: -0.4422581195831299
train_loss: -0.39866940180460614
train_loss: -0.3487079938252767
train_loss: -0.362565557161967
train_loss: -0.3070661822954814
train_loss: -0.3743214209874471
train_loss: -0.30561800797780353
train_loss: -0.39610175291697186
train_loss: -0.3550885121027629
train_loss: -0.35532716910044354
train_loss: -0.3973332643508911
train_loss: -0.4144953489303589
Current Best ND is: 0.62594
Epoch 28/100
train_loss: -0.32663458585739136
train_loss: -0.43190352121988934
train_loss: -0.3609148661295573
train_loss: -0.38879791895548504
train_loss: -0.3407641251881917
train_loss: -0.38636688391367596
train_loss: -0.43711618582407635
train_loss: -0.3913713296254476
train_loss: -0.3707411289215088
train_loss: -0.3624272346496582
train_loss: -0.3533920844395955
train_loss: -0.4011656443277995
train_loss: -0.4184991518656413
train_loss: -0.3745376666386922
train_loss: -0.37162025769551593
train_loss: -0.40406835079193115
train_loss: -0.37737

KeyboardInterrupt: 